In [1]:
import pandas as pd
import pickle

In [2]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sivateja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data=pd.read_csv('spam_ham_dataset.csv')


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [15]:
for i in range(0,data.shape[0]):
    data['text'][i]=data['text'][i].replace('Subject:', '')
data.head()

,label,text
0,ham,enron methanol ; meter # : 988291\r\nthis is ...
1,ham,"hpl nom for january 9 , 2001\r\n( see attache..."
2,ham,"neon retreat\r\nho ho ho , we ' re around to ..."
3,spam,"photoshop , windows , office . cheap . main t..."
4,ham,re : indian springs\r\nthis deal is to book t...


In [16]:
data=data[['label','text']]
data['text'][0]

" enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

Messages cleaning

In [17]:

cleaned_message_2=[]
porter = PorterStemmer()
wordnet = WordNetLemmatizer()

In [18]:
for i in range(0,data.shape[0]):
    # Remove all the special characters
    processed_message = re.sub(r'\W', ' ', str(data.text[i]))

    # remove all single characters
    processed_message= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_message)

    # Remove single characters from the start
    processed_message = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_message) 

    # Substituting multiple spaces with single space
    processed_message = re.sub(r'\s+', ' ', processed_message, flags=re.I)

    # Removing prefixed 'b'
    processed_message = re.sub(r'^b\s+', '', processed_message)

    # Converting to Lowercase
    processed_message = processed_message.lower()
    
    words=processed_message.split()
    
    #Lemmatize the words
    words = [wordnet.lemmatize(word,pos="v") for word in words]
    
    #Removing stop words
    words = [word for word in words if word not in set(stopwords.words('english'))]
    
    #Joing the words
    processed_message = ' '.join(words)
    
    #Adding this to cleaned meassage
    cleaned_message_2.append(processed_message)



 

In [19]:
cleaned_message_2[0]

'enron methanol meter 988291 follow note give monday 4 3 00 preliminary flow data provide daren please override pop daily volume presently zero reflect daily activity obtain gas control change need asap economics purpose'

Feature Generation using Bag of Words

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000)

X= cv.fit_transform(cleaned_message_2).toarray()

In [22]:
# Extracting dependent variable from the dataset
y = pd.get_dummies(data['label'])
y = y.iloc[:, 1].values

In [23]:
# Creating a pickle file for the CountVectorizer
pickle.dump(cv, open('cv-transform_email.pkl', 'wb'))


In [29]:
# Model Building

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
from sklearn import metrics
predicted= classifier.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.9373549883990719


In [31]:
# Creating a pickle file for the Multinomial Naive Bayes model
filename = 'spam_detection_model_email.pkl'
pickle.dump(classifier, open(filename, 'wb'))